In [86]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import multiCLFutils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch

In [87]:
DATA_PATH='../../../수업/TORCH_IMAGE/data/'
TRAIN_FILE='mnist_train.csv'
TEST_FILE='mnist_test.csv'
trainDF=pd.read_csv(DATA_PATH+TRAIN_FILE,header=None)
testDF=pd.read_csv(DATA_PATH+TEST_FILE,header=None)
train=trainDF[trainDF.columns[:-1]]
label=trainDF[trainDF.columns[[-1]]]

test=testDF[testDF.columns[:-1]]
test_label=testDF[testDF.columns[[-1]]]

In [88]:
train.shape,label.shape

((60000, 784), (60000, 1))

In [89]:
trainDS=utils.MyDataSet(train,label)
train_loader=DataLoader(trainDS,batch_size=100)

testDS=utils.MyDataSet(test,test_label)
test_loader=DataLoader(testDS,batch_size=100)

In [90]:
class FasionCNN(nn.Module):
    def __init__(self):
        super(FasionCNN,self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer2=nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1=nn.Linear(in_features=64*6*6,out_features=600)
        self.drop=nn.Dropout2d(0.25)
        self.fc2=nn.Linear(in_features=600,out_features=120)
        self.fc3=nn.Linear(in_features=120,out_features=10)

    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        out=self.drop(out)
        out=self.fc2(out)
        out=self.fc3(out)
        return out

In [91]:
learning_rate=0.001
model=FasionCNN()

In [92]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)
print(model)

FasionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [96]:
new_epochs=5
count=0
loss_list=[]
iteration_list=[]
accuracy_list=[]

predictions_list=[]
labels_list=[]

for epoch in range(new_epochs):
    for images,labels in train_loader:

        train=Variable(images.view(100,1,28,28))
        labels=Variable(labels)

        outputs=model(train)
        loss=criterion(outputs,labels.reshape(-1).long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count+=1

        if not (count % 50):
            total=0
            correct=0
            for images,labels in test_loader:
                labels_list.append(labels)
                test=Variable(images.view(100,1,28,28))
                outputs=model(test)
                predictions=torch.max(outputs,1)[1]
                predictions_list.append(predictions)
                correct+=(predictions==labels).sum()
                total+=len(labels)

            accuracy=correct*100/total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)

        if not (count % 500) :
            print(f'Iteration : {count}, Loss : {loss.data}, Accuracy : {accuracy}%')

c:\Users\KDP-30\anaconda3\envs\TORCH_38\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Iteration : 500, Loss : 0.15700212121009827, Accuracy : 1058.3399658203125%
Iteration : 1000, Loss : 0.17212453484535217, Accuracy : 1061.1500244140625%
Iteration : 1500, Loss : 0.049235425889492035, Accuracy : 1061.8699951171875%
Iteration : 2000, Loss : 0.003647630801424384, Accuracy : 1062.3299560546875%
Iteration : 2500, Loss : 0.002935508731752634, Accuracy : 1061.739990234375%
Iteration : 3000, Loss : 0.2826916575431824, Accuracy : 1062.3599853515625%
